<h1 style="color:green">NBU-powered Scratchpad</h1>
<p style="color:green">This notebook is used for code smithening purposes</p>
<p style="color:green">It includes notebook utilities support, which means it can easily invoke functionality from the Conway-based project it belongs to</p>
<p>NB: this notebook requires to be launched in an environment that includes conway and conway.ops</p>

In [1]:
import os                                                               as _os
import sys
sys.path.append(_os.path.dirname(_os.path.dirname(_os.getcwd())) + "/src") # So we can import scratch_nb_apps
from scratch_nb_apps.scratch_nb_utils import Scratch_NB_Utils
NBU                       = Scratch_NB_Utils()
DFU                       = NBU.DFU
T0                        = NBU.time.perf_counter()

SCRATCH installation:                scratch_fork    
Jupyter using repo[branch]:      scratch.ops[afin-dev]    
Installation path:               /home/alex/consultant1@CCL/dev/scratch_fork    
Application:                     <class 'scratch_nb_apps.scratch_nb_application.Scratch_NB_Application'>    


<h1 style="color:purple">Create more branches</h1>

In [5]:
p = NBU.Application.app().profile

AttributeError: 'Scratch_NB_Application' object has no attribute 'profile'

In [4]:
setup = NBU.RepoSetup(sdlc_root = "/home/alex/admin1@CCL/dev/sdlc", profile_name="consultant1@FIN", sdlc_project = "sdlc")

ValueError: Unable to load TOML file '/home/alex/admin1@CCL/dev/sdlc/sdlc.profiles/consultant1@FIN/profile.toml'. Error is:
	[Errno 2] No such file or directory: '/home/alex/admin1@CCL/dev/sdlc/sdlc.profiles/consultant1@FIN/profile.toml'

<h1 style="color:purple">Graph of Awaitables</h1>
<p> We pilot whether we can maintain a graph of awaitables, including:</p>
<li> Multiple dependencies: awaitables B, C both awaiting on a common awaitable A

In [2]:
import asyncio

In [3]:
class UsheringTo():

    def __init__(self, result_l):
        self.result_l = result_l
        #self._to_do = []

    def __iadd__(self, coro):
        if not asyncio.iscoroutine(coro):
            raise ValueError(f"Expected a coroutine, but instead got a {type(coro)}")
        self.to_do.append(coro)
        return self

    async def __aenter__(self):
        self.to_do = []
        return self

    async def __aexit__(self, exc_type, exc_value, traceback):
        to_do_iter                                      = asyncio.as_completed(self.to_do)
        for coro in to_do_iter:
            coro_result                                 = await coro
            self.result_l.append(coro_result)
        return

In [4]:
class MachineBase():
    def __init__(self, verbose, compute_time):
        self.verbose = verbose
        self.compute_time = compute_time

    async def compute(self, path):
        return await asyncio.sleep(self.compute_time)
    
    async def run_node(self, node_name, node_callable, prefix="") -> str:
        name = f"{prefix}{node_name}"
        if self.verbose:
            print(f"{name} ...", flush=True)
        x = await node_callable(f"{name}->")
        if self.verbose:
            print(f"... {name}", flush=True)
        return f"{name}"

class Machine1(MachineBase):
    
    def __init__(self, verbose=False, compute_time=1):
        super().__init__(verbose, compute_time)
        
    async def A(self, prefix="") -> str:
        return await self.run_node("A", self.compute, prefix)

    async def B(self, prefix="") -> str:
        return await self.run_node("B", self.A, prefix)

    async def C(self, prefix="") -> str:
        return await self.run_node("C", self.B, prefix)



In [5]:
async def supervisor(to_do_l, nb_tries = 1, verbose=False):
    result_l                                        =  []

    async with UsheringTo(result_l) as usher:
        for coro in to_do_l:
            usher += coro
    return result_l

In [17]:
COMPUTE_TIME = 0.5 # (in seconds)
m1 = Machine1(compute_time=COMPUTE_TIME)

In [23]:
#result_l = asyncio.run(supervisor()) # Don't do this in Jupyter since Jupyter is already running an event loop
NB_TRIES = 5000
TO_DO_L = []
for idx in range(NB_TRIES):
    TO_DO_L += [m1.A(), m1.B(), m1.C()]
with NBU.Profiler("2 levels test"):
        result_l = await supervisor(TO_DO_L, nb_tries=NB_TRIES)
len(result_l), f"{len(result_l)*COMPUTE_TIME/60/60} hr of wait"


[325.90 sec]	interactiveshell:3551	2 levels test completed in 0.78 sec


(15000, '2.0833333333333335 hr of wait')